# 3D Photography using Context-aware Layered Depth Inpainting (CVPR 2020)

[project website](https://shihmengli.github.io/3D-Photo-Inpainting/)

> Colab adapted version by [Manuel Romero/@mrm8488](https://twitter.com/mrm8488)

## 1. Install required packages


In [1]:
!pip3 install -q torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
!pip3 install -q opencv-python==4.2.0.32
!pip3 install -q vispy==0.6.4
!pip3 install -q moviepy==1.0.2
!pip3 install -q transforms3d==0.3.1
!pip3 install -q networkx==2.3
!pip install -q PyDrive

     |████████████████████████████████| 723.9 MB 24 kB/s 
     |████████████████████████████████| 4.0 MB 36.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.4.0+cu100 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.4.0+cu100 which is incompatible.
     |████████████████████████████████| 28.2 MB 51.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 2.3 MB 12.5 MB/s 
     |████████████████████████████████| 890 kB 50.6 MB/s 
     |████████████████████████████████| 7.8 MB 8.9 MB/s 
   

## 2. Download script and pretrained model
- It will ask for you a verification code


In [2]:
![ -e 3d-photo-inpainting-master ] && rm -r 3d-photo-inpainting-master

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id':"1Jmiq1-hSIsHiq_GCcmiNXtrs_T62AY_z"})
downloaded.GetContentFile('3d-photo-inpainting-master.zip')

In [3]:
%%bash
unzip 3d-photo-inpainting-master.zip
rm 3d-photo-inpainting-master.zip
[ -e /content/3d-photo-inpainting-master/image/moon.jpg ] && rm -rf /content/3d-photo-inpainting-master/image/moon.jpg

Archive:  3d-photo-inpainting-master.zip
   creating: 3d-photo-inpainting-master/
  inflating: 3d-photo-inpainting-master/LICENSE  
   creating: 3d-photo-inpainting-master/MiDaS/
  inflating: 3d-photo-inpainting-master/MiDaS/MiDaS_utils.py  
  inflating: 3d-photo-inpainting-master/MiDaS/monodepth_net.py  
  inflating: 3d-photo-inpainting-master/MiDaS/run.py  
  inflating: 3d-photo-inpainting-master/MiDaS/model.pt  
   creating: 3d-photo-inpainting-master/depth/
  inflating: 3d-photo-inpainting-master/depth/moon.npy  
   creating: 3d-photo-inpainting-master/image/
  inflating: 3d-photo-inpainting-master/image/moon.jpg  
  inflating: 3d-photo-inpainting-master/mesh_tools.py  
  inflating: 3d-photo-inpainting-master/networks.py  
   creating: 3d-photo-inpainting-master/misc/
  inflating: 3d-photo-inpainting-master/misc/moon_40.gif  
  inflating: 3d-photo-inpainting-master/README.md  
  inflating: 3d-photo-inpainting-master/main.py  
   creating: 3d-photo-inpainting-master/video/
   creati

In [4]:
cd 3d-photo-inpainting-master

/content/3d-photo-inpainting-master


In [11]:
# Disable PLY generation
!sed -i 's/save_ply: True/save_ply: False/' argument.yml

In [12]:
# Switch off screen rendering
!sed -i 's/offscreen_rendering: True/offscreen_rendering: False/g' argument.yml

## 3. Upload a picture from your filesystem with jpg extension

In [7]:
!rm -rf /content/3d-photo-inpainting-master/image/*

In [8]:
import os
import shutil
from google.colab import files

uploaded = files.upload()
pic_name = list(uploaded.keys())[0]

if(pic_name.split(".")[1].lower() != "jpg"):
  print("YOU MUST UPLOAD A .jpg FILE!! Re-run this cell and upload a pic with jpg extension")
  os.remove("./" + pic_name)
else:
  shutil.move("./" + pic_name, "/content/3d-photo-inpainting-master/image/" + pic_name)

Saving init.jpg to init.jpg


## 4. Execute the 3D Photo Inpainting
  - Note: The 3D photo generation process usually takes about 2-3 minutes or more depending on the available computing resources.

In [13]:
%%time
!python main.py --config argument.yml

  0% 0/1 [00:00<?, ?it/s]Current Source ==>  init
initialize
device: cpu
start processing
  processing image/init.jpg (1/1)
torch.Size([1, 3, 256, 384])
finished
Start Running 3D_Photo ...
53.13010235415598
^C
CPU times: user 1.49 s, sys: 413 ms, total: 1.9 s
Wall time: 3min 48s


- **The results are stored in the following directories**
  - Corresponding depth map estimated by [MiDaS](https://github.com/intel-isl/MiDaS.git) 
      - E.g. ```/content/3d-photo-inpainting-master/depth/moon.npy```
  - Inpainted 3D mesh
      - E.g. ```/content/3d-photo-inpainting-master/mesh/moon.ply```
  - Rendered videos with circular motion
      - E.g. ```/content/3d-photo-inpainting-master/mesh/moon.mp4```

## 5. Visualize results
### 5.1 Swing effect

In [14]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/3d-photo-inpainting-master/video/' + pic_name.split(".")[0] +  '_swing.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=600 controls autoplay>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

### 5.2 Straight line effect

In [ ]:
mp4 = open('/content/3d-photo-inpainting-master/video/' + pic_name.split(".")[0] +  '_straight-line.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=600 controls autoplay>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Download it!
### Download all!

In [ ]:
directory = '/content/3d-photo-inpainting-master/video'
for filename in os.listdir(directory):
    if (filename.endswith(".mp4")): 
         files.download(os.path.join(directory, filename))


### Download the one with the effect you want (or both)

In [ ]:
files.download('/content/3d-photo-inpainting-master/video/' + pic_name.split(".")[0] +  '_swing.mp4')

In [ ]:
files.download('/content/3d-photo-inpainting-master/video/' + pic_name.split(".")[0] +  '_straight-line.mp4')

### Download 3D generated file (PLY)

In [ ]:
files.download('/content/3d-photo-inpainting-master/mesh/' + + pic_name.split(".")[0] + '.ply')

### If you want to try with another picture go to step 3

<p>Did you enjoy? Buy me a coffe :)</p>
<p><a href="https://ko-fi.com/Y8Y3VYYE"><img src="https://camo.githubusercontent.com/c8a9d4f1653d599167ef09852550c6810a7306bc/68747470733a2f2f7777772e6b6f2d66692e636f6d2f696d672f676974687562627574746f6e5f736d2e737667" alt="ko-fi" data-canonical-src="https://www.ko-fi.com/img/githubbutton_sm.svg"></a></p>